__Решение задачи об укладке рюкзака с помощью собственного алгоритма__

Варианты заданий: 1.2, 2.2, 3.2, 4.3, 5.3
* жадный выбор, начиная со случайного груза
* выбрать только 20% самых приспособленных особей
* однородный (каждый бит от случайно выбранного родителя)
* добавление 1 случайной вещи 10% особей
* замена своих родителей

In [1]:
import json
import random as rnd

data_path = './15.txt'
result_path = './result 2.json'

__Чтение данных из файла__

In [2]:
def load_data():
    f = open(data_path)
    
    
    first_line = f.readline().split(' ')
    bag_data = {'max_weight': 0, 'max_volume': 0, 'items': []}
    bag_data['max_weight'] = int(first_line[0])
    bag_data['max_volume'] = float(first_line[1])
    
    for line in f:
        values = [x for x in line.split(' ')]
        bag_data['items'].append({'weight': int(values[0]), 'volume': float(values[1]), 'price': int(values[2])})
        
    return bag_data

__Функция приспособления__

In [3]:
def fitness(individual, items):
    weight, volume, price = 0, 0, 0
    for (selected, item) in zip(individual, items):
        if selected:
            weight += item['weight']
            volume += item['volume']
            price += item['price']

    if weight > max_weight or volume > max_volume:
        price = 0

    return price

__Поиск лучшей особи в популяции на основе функции приспособления__

In [4]:
def get_best_individual(population, items):
    candidates = []

    for ind in population:
        candidates.append((fitness(ind, items), ind))

    best_ind = candidates[0]
    for candidate in candidates:
        best_ind = candidate if candidate[0] > best_ind[0] else best_ind

    return best_ind

__Создание популяции жадным выбором__

In [37]:
def create_individual(items):
    individual = [0 for x in range(len(items))]
    
    total_weight = 0
    total_volume = 0
    
    draw_order = [i for i in range(len(items))]
    rnd.shuffle(draw_order)
    
    for index in draw_order:
        item = items[index]
        
        if total_weight + item['weight'] > max_weight or total_volume + item['volume'] > max_volume:
            continue
        else:
            total_weight += item['weight']
            total_volume += item['volume']
            individual[index] = 1
    
    return individual

def create_population(items, count = 200):
    return [create_individual(items) for i in range(count)]

__Отбор 20% самых приспособленных особей__

In [6]:
def choose_to_crossover(population, items, percent = 20):
    count = int(len(population) * percent / 100)
    #Округление до чётного нужно, чтобы все родители скрестились
    count += count % 2
    
    fitnessed_population = [(ind, fitness(ind, items)) for ind in population]
    fitnessed_population = sorted(population, key = lambda i: i[1], reverse = True)
    
    return fitnessed_population[0:count]

__Однородное скрещивание__

In [27]:
def crossover_parents(parent1, parent2, child_count = 2):
    parents = [parent1, parent2]
    children = []
    length = len(parent1)
    
    for _ in range(child_count):
        child = [parents[rnd.randrange(2)][i] for i in range(length)]
        children.append(child)
        
    return children

def crossover(population, items):
    children = []
    parents = [i for i in population]
    while len(parents) != 0:
        parent1 = rnd.choice(parents)
        parents.remove(parent1)
        parent2 = rnd.choice(parents)
        parents.remove(parent2)
        
        children.extend(crossover_parents(parent1, parent2))
        
    return children

__Мутация (добавление 1 случайной вещи 10% потомков)__

In [8]:
#Если при любых изменениях особь умирает, то мутация не происходит
def mutation_individual(individual, items):
    unselected_indexes = [index for index, selected in enumerate(individual) if selected == 0]
    while len(unselected_indexes) != 0:
        index = rnd.choice(unselected_indexes)
        individual[index] = 1
        if fitness(individual, items) == 0:
            individual[index] = 0
            unselected_indexes.remove(index)
        else:
            return
        
def mutation(population, items):
    for individual in population:
        mutation_individual(individual, items)

__Обновление популяции (все родители замещаются их детьми)__

In [9]:
def update_population(population, parents, children):
    new_population = []
    for individual in population:
        if individual in parents:
            new_population.append(children.pop())
            parents.remove(individual)
        else:
            new_population.append(individual)
    return new_population

In [32]:
def ga():
    population = create_population(items)

    for g in range(500):
        parents = choose_to_crossover(population, items)

        children = crossover(parents, items)

        mutation(children, items)

        population = update_population(population, parents, children)

    return get_best_individual(population, items)

__Перевод "гена" в конкретные предметы и вывод результата__

In [42]:
bag_data = load_data()
max_weight = bag_data['max_weight']
max_volume = bag_data['max_volume']
items = bag_data['items']

gene = ga()[1]
selected_items = [items[index] for index, bit in enumerate(gene) if bit == 1]

res = {
    'weight': sum([item['weight'] for item in selected_items]),
    'volume': sum([item['volume'] for item in selected_items]),
    'price': sum([item['price'] for item in selected_items]),
    'items': selected_items
}

with open(result_path, 'w') as file:
    json.dump(res, file, indent=2)
    
print(res)

{'weight': 11695, 'volume': 12.0, 'price': 4098, 'items': [{'weight': 486, 'volume': 0.4, 'price': 385}, {'weight': 182, 'volume': 0.6, 'price': 283}, {'weight': 839, 'volume': 1.0, 'price': 385}, {'weight': 1390, 'volume': 0.4, 'price': 159}, {'weight': 425, 'volume': 0.6, 'price': 174}, {'weight': 1082, 'volume': 0.7, 'price': 186}, {'weight': 1119, 'volume': 0.5, 'price': 264}, {'weight': 1173, 'volume': 1.0, 'price': 351}, {'weight': 809, 'volume': 1.2, 'price': 336}, {'weight': 779, 'volume': 1.0, 'price': 180}, {'weight': 415, 'volume': 0.6, 'price': 180}, {'weight': 554, 'volume': 1.0, 'price': 377}, {'weight': 1084, 'volume': 0.5, 'price': 112}, {'weight': 685, 'volume': 0.4, 'price': 103}, {'weight': 325, 'volume': 1.1, 'price': 377}, {'weight': 348, 'volume': 1.0, 'price': 246}]}
